In [59]:
import pandas as pd
import numpy as np
import keras as kr
import interpret

data = pd.read_csv('earthquakes_scaled_class.csv', index_col="index")

data = data.drop("index.1", axis=1)

data

,magnitude,time,updated,felt,cdi,mmi,tsunami,sig,nst,dmin,...,gap,magType,depth,latitude,longitude,distanceKM,continent,country,subnational,city
index,,,,,,,,,,,,,,,,,,,,,
us6000lcz2,5,-0.815799,-0.977412,-0.083333,-0.6,-1.0,0.0,0.000000,0.491667,1.290504,...,0.105263,0.25,0.000000,-0.126122,0.902897,-0.804348,0.5,0.275862,0.129630,0.234432
us6000n9xs,4,0.289580,0.000423,-0.083333,-0.6,0.0,0.0,-0.919014,-0.608333,-0.232505,...,1.526316,-0.75,0.001914,0.000000,-0.153676,0.369565,0.0,0.000000,0.000000,0.000000
tx2024mydx,4,0.289510,0.349996,-0.083333,-0.6,0.0,0.0,-0.838028,-0.583333,-0.295730,...,0.315789,-0.75,-0.149192,0.000098,-0.153676,0.369565,0.0,0.000000,0.000000,0.000000
us6000n9xf,4,0.289510,0.000343,-0.083333,-0.6,0.0,0.0,-0.799296,-0.608333,-0.228936,...,1.657895,-0.75,-0.043131,0.000601,-0.153707,0.391304,0.0,0.000000,0.000000,0.000000
nn00880145,4,0.278268,0.311118,-0.083333,-0.6,-1.0,0.0,-0.919014,-0.775000,0.137667,...,0.022105,-0.75,-0.092003,0.580415,-0.207513,-0.086957,0.0,0.000000,-0.666667,-2.021978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ci40731623,5,0.580644,0.377703,774.583333,0.4,1.0,1.0,1.299296,0.358333,-0.330208,...,1.368421,-0.50,0.049682,0.180617,-0.216912,-0.673913,0.0,0.000000,-1.555556,-0.952381
ci40699207,4,0.455449,0.368965,1009.375000,0.4,1.0,1.0,1.362676,0.750000,-0.329897,...,-0.657895,-0.50,0.036065,0.181669,-0.214216,-0.717391,0.0,0.000000,-1.555556,-0.952381
ci40699207,4,0.455449,0.368965,1009.375000,0.4,1.0,1.0,1.362676,0.750000,-0.329897,...,-0.657895,-0.50,0.036065,0.181669,-0.214216,-0.717391,0.0,0.000000,-1.555556,-0.952381


In [60]:
data["magnitude"] = data["magnitude"].map(lambda x: x - 3)
data["magnitude"].unique()

array([2, 1, 0, 3, 4, 5], dtype=int64)

In [61]:
variables_despues_analisis = [
    "felt",         # Número de personas que reportan haber sentido el terremoto
    "cdi",          # Community Determined Intensity
    "mmi",          # Modified Mercalli Intensity
    "alert",        # Nivel de alerta
    "status",       # Estado revisado o automático
    "sig",          # Significancia del terremoto
    "ids",          # IDs de eventos relacionados
    "sources",      # Fuentes de los datos
    "types",        # Tipos de eventos asociados
    "nst",          # Número de estaciones que registraron el evento
    "dmin",         # Distancia mínima al evento
    "rms",          # RMS de la señal
    "gap",          # Brecha de datos entre estaciones
    "magType",      # Tipo de magnitud
    "distanceKM",   # Distancia a áreas pobladas en kilómetros
    "placeOnly",    # Si solo se tiene información del lugar
    "locationDetails" # Detalles adicionales de la ubicación
    "sig"
]

for var in variables_despues_analisis:
    try:
        data = data.drop(var, axis=1)
    except:
        pass

data

,magnitude,time,updated,tsunami,depth,latitude,longitude,continent,country,subnational,city
index,,,,,,,,,,,
us6000lcz2,2,-0.815799,-0.977412,0.0,0.000000,-0.126122,0.902897,0.5,0.275862,0.129630,0.234432
us6000n9xs,1,0.289580,0.000423,0.0,0.001914,0.000000,-0.153676,0.0,0.000000,0.000000,0.000000
tx2024mydx,1,0.289510,0.349996,0.0,-0.149192,0.000098,-0.153676,0.0,0.000000,0.000000,0.000000
us6000n9xf,1,0.289510,0.000343,0.0,-0.043131,0.000601,-0.153707,0.0,0.000000,0.000000,0.000000
nn00880145,1,0.278268,0.311118,0.0,-0.092003,0.580415,-0.207513,0.0,0.000000,-0.666667,-2.021978
...,...,...,...,...,...,...,...,...,...,...,...
ci40731623,2,0.580644,0.377703,1.0,0.049682,0.180617,-0.216912,0.0,0.000000,-1.555556,-0.952381
ci40699207,1,0.455449,0.368965,1.0,0.036065,0.181669,-0.214216,0.0,0.000000,-1.555556,-0.952381
ci40699207,1,0.455449,0.368965,1.0,0.036065,0.181669,-0.214216,0.0,0.000000,-1.555556,-0.952381


In [62]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.15)

train_x = train.drop("magnitude", axis=1)
train_y = train["magnitude"]

test_x = test.drop("magnitude", axis=1)
test_y = test["magnitude"]

In [63]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier(interactions=20, inner_bags=10, outer_bags=60)
ebm.fit(train_x, train_y)

C:\Users\mario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\interpret\glassbox\_ebm\_ebm.py:1014: UserWarning:

Detected multiclass problem. Forcing interactions to 0. Multiclass interactions only have local explanations. They are not currently displayed in the global explanation visualizations. Set interactions=0 to disable this warning. If you still want multiclass interactions, this API accepts a list, and the measure_interactions function can be used to detect them.



ExplainableBoostingClassifier(inner_bags=10, interactions=20, outer_bags=60)

In [64]:
from interpret import show

show(ebm.explain_global())

<!-- http://127.0.0.1:7001/2682469744592/ -->

In [65]:
from sklearn.metrics import classification_report

predicciones = ebm.predict(test_x).astype(float)

print(classification_report(test_y, predicciones))

              precision    recall  f1-score   support

           0       1.00      0.78      0.88        23
           1       0.84      0.96      0.90        49
           2       0.76      0.52      0.62        42
           3       0.74      0.91      0.81        53
           4       1.00      0.75      0.86         4

    accuracy                           0.81       171
   macro avg       0.87      0.78      0.81       171
weighted avg       0.81      0.81      0.80       171



TRAIN FULL DATASET. 

In [72]:
#label 0: 0-2
#label 1: 3-5

data["magnitude1"] = data["magnitude"].map(lambda x: 1 if x >= 3 else 0)
data["magnitude2"] = data["magnitude"].map(lambda x: 1 if x == 5 or x == 0 else 0)
data["magnitude3"] = data["magnitude"].map(lambda x: 1 if x == 4 or x == 2 else 0)

data

,magnitude,time,updated,tsunami,depth,latitude,longitude,continent,country,subnational,city,magnitude1,magnitude2,magnitude3
index,,,,,,,,,,,,,,
us6000lcz2,2,-0.815799,-0.977412,0.0,0.000000,-0.126122,0.902897,0.5,0.275862,0.129630,0.234432,0,0,1
us6000n9xs,1,0.289580,0.000423,0.0,0.001914,0.000000,-0.153676,0.0,0.000000,0.000000,0.000000,0,0,0
tx2024mydx,1,0.289510,0.349996,0.0,-0.149192,0.000098,-0.153676,0.0,0.000000,0.000000,0.000000,0,0,0
us6000n9xf,1,0.289510,0.000343,0.0,-0.043131,0.000601,-0.153707,0.0,0.000000,0.000000,0.000000,0,0,0
nn00880145,1,0.278268,0.311118,0.0,-0.092003,0.580415,-0.207513,0.0,0.000000,-0.666667,-2.021978,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ci40731623,2,0.580644,0.377703,1.0,0.049682,0.180617,-0.216912,0.0,0.000000,-1.555556,-0.952381,0,0,1
ci40699207,1,0.455449,0.368965,1.0,0.036065,0.181669,-0.214216,0.0,0.000000,-1.555556,-0.952381,0,0,0
ci40699207,1,0.455449,0.368965,1.0,0.036065,0.181669,-0.214216,0.0,0.000000,-1.555556,-0.952381,0,0,0


PREDICT BETWEEN 0-2 OR 3-5

In [76]:
train, test = train_test_split(data, test_size=0.15)

train_x = train.drop(["magnitude1", "magnitude", "magnitude2", "magnitude3"], axis=1)
train_y = train["magnitude1"]

test_x = test.drop(["magnitude1", "magnitude", "magnitude2", "magnitude3"], axis=1)
test_y = test["magnitude1"]

ebm = ExplainableBoostingClassifier(interactions=20, inner_bags=10, outer_bags=60)

ebm.fit(train_x, train_y)

ExplainableBoostingClassifier(inner_bags=10, interactions=20, outer_bags=60)

In [77]:
predicciones = ebm.predict(test_x).astype(float)

print(classification_report(test_y, predicciones))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93       114
           1       0.86      0.88      0.87        57

    accuracy                           0.91       171
   macro avg       0.90      0.90      0.90       171
weighted avg       0.91      0.91      0.91       171



In [78]:
show(ebm.explain_global())

<!-- http://127.0.0.1:7001/2682531429392/ -->

PREDICT TSUNAMI

In [66]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.15)

train_x = train.drop("tsunami", axis=1)
train_y = train["tsunami"]

test_x = test.drop("tsunami", axis=1)
test_y = test["tsunami"]

In [67]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier(interactions=20, inner_bags=10, outer_bags=60)
ebm.fit(train_x, train_y)

ExplainableBoostingClassifier(inner_bags=10, interactions=20, outer_bags=60)

In [68]:
from sklearn.metrics import classification_report

predicciones = ebm.predict(test_x).astype(float)

print(classification_report(test_y, predicciones))

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       160
         1.0       1.00      0.73      0.84        11

    accuracy                           0.98       171
   macro avg       0.99      0.86      0.92       171
weighted avg       0.98      0.98      0.98       171



In [69]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_y, predicciones)

array([[160,   0],
       [  3,   8]], dtype=int64)

In [70]:
from interpret import show

show(ebm.explain_global())

<!-- http://127.0.0.1:7001/2682469838480/ -->

In [71]:
data.columns

Index(['magnitude', 'time', 'updated', 'tsunami', 'depth', 'latitude',
       'longitude', 'continent', 'country', 'subnational', 'city'],
      dtype='object')